# Context maintenance strategies

As agents interact with users and environments, the context they maintain must evolve to reflect new information. The strategy for updating context - whether to append new information, overwrite existing facts, version changes or intelligently merge updates - has profound implications for correctness, token efficiency and the ability to track how information changes over time. Each strategy represents different trade-offs between history preservation, storage efficiency and retrieval simplicity.

Context maintenance is not a one-size-fits-all problem. Conversation histories benefit from append-only strategies that preserve every turn in order. User preferences might use update-in-place to maintain only current values. Important configuration might need versioning to track what changed and when. Multi-source data integration requires structured merging with conflict resolution. Understanding these strategies and their appropriate applications is essential for building agents that maintain accurate, efficient context.

In this notebook, we explore the four primary context maintenance strategies and when to apply each. We will examine append-only approaches for preserving complete history, update-in-place strategies for minimal storage, versioned updates that track changes over time, and structured merging for intelligent conflict resolution. We will compare their trade-offs and learn to combine strategies into hybrid approaches that optimize for different types of information.

In [1]:
import os
from typing import List, Dict, Optional, Any
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
from copy import deepcopy
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import json
import time

### Initialize the language model

In [2]:
# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY", "").strip(),
    temperature=0  # Set to 0 for more deterministic outputs
)

## Part 1: Append-only strategy

The append-only strategy never modifies or deletes existing context - it only adds new entries. This preserves complete history with perfect fidelity, maintaining temporal ordering and enabling full reconstruction of how context evolved. Every conversation turn, every event, every update gets recorded as a new entry in a growing log. This approach is invaluable when history matters, when audit trails are required, or when we need to understand the sequence of events that led to a current state.

Think of append-only as maintaining a journal where we never erase anything - qw only add new pages. When a user changes their mind about something, we don't go back and cross out the old preference; we simply write down the new one with a timestamp. This means we can always go back and see exactly what was said when. For conversations, this is natural: we would not delete parts of a chat history just because the topic changed. For events like purchases or updates, having the complete timeline helps with customer support, debugging and analytics. The append-only pattern mirrors how Git works with commits - nothing is ever truly deleted, only added.

The trade-off is straightforward: append-only strategies consume storage and tokens linearly with the number of updates. As context grows, we must either retrieve selectively - taking only recent entries - or summarize historical context. The benefit is simplicity and completeness. Nothing is ever lost, contradictions are preserved rather than hidden, and we always have the raw material to understand what happened and when. When a customer service agent asks "what did this user tell us three interactions ago?", append-only gives us the exact answer immediately.

### Defining the basic append-only structure
To implement append-only context maintenance, we need a way to represent individual pieces of information as discrete entries in a timeline. Each entry should capture not just the content itself, but also essential metadata that helps us understand the context of that information. The timestamp tells us when something was added, which is critical for temporal ordering and understanding the sequence of events. The entry type helps us categorize different kinds of information - is this a user message, an agent response, a system event, or something else? This categorization becomes powerful for filtering and retrieval later. Finally, we need a flexible metadata dictionary to store any additional context-specific information like source, confidence levels, user IDs, session identifiers, or any other attributes that might be relevant for specific use cases. This foundational structure gives us everything needed to build a complete, immutable history of context evolution.

In [3]:
@dataclass
class ContextEntry:
    """Single entry in append-only context log."""
    
    # When this entry was created - critical for temporal ordering
    timestamp: datetime
    
    # Type of entry: user_message, agent_message, system_event, etc.
    # Helps categorize and filter entries
    entry_type: str
    
    # The actual content of this entry
    content: str
    
    # Additional metadata for rich context
    # Can store things like: source, confidence, user_id, session_id
    metadata: Dict[str, Any] = field(default_factory=dict)

Now let's build the `AppendOnlyContext` class that maintains a growing list of these entries and provides methods to retrieve them. The key principle is that the entries list is immutable in the sense that existing entries are never modified or deleted - we only append new ones. This class needs to support two primary retrieval patterns: getting the complete history for audit or debugging purposes, and getting recent history for efficiency when passing context to an LLM. The recent history pattern is particularly important because it allows us to manage token limits by selecting just the most relevant recent entries while keeping the full history available when needed.

In [4]:
class AppendOnlyContext:
    """Append-only context maintenance - never modifies or deletes existing entries."""
    
    def __init__(self):
        # Main storage: chronologically ordered list of all entries
        # This list only grows, never shrinks or modifies existing entries
        self.entries: List[ContextEntry] = []  # Each element is a ContextEntry object with timestamp, type, content, metadata
    
    def append(self, entry_type: str, content: str, metadata: Dict = None):
        """
        Add new entry to context (never modifies existing entries).
        
        Args:
            entry_type: Category of entry (e.g., "user_message", "agent_message", "system_event")
            content: The actual content/text of this entry
            metadata: Optional dict for additional context (source, user_id, confidence, etc.)
        """
        # Create a new entry with current timestamp - captures the exact moment of creation
        entry = ContextEntry(
            timestamp=datetime.now(),
            entry_type=entry_type,
            content=content,
            metadata=metadata or {}  # Default to empty dict if no metadata provided
        )
        # Append to the end - preserving chronological order. This is the only mutation operation - no updates or deletes allowed
        self.entries.append(entry)
    
    def get_full_context(self) -> str:
        """
        Get complete context as formatted string.
        Useful for debugging, audit trails, and full history reconstruction.
        """
        lines = []
        # Iterate through all entries in chronological order (oldest to newest)
        for entry in self.entries:
            # Format timestamp for human readability (hour:minute:second)
            timestamp_str = entry.timestamp.strftime('%H:%M:%S')
            # Format: [timestamp] type: content
            lines.append(f"[{timestamp_str}] {entry.entry_type}: {entry.content}")
        # Join all lines with newlines for readable multi-line output
        return "\n".join(lines)
    
    def get_recent_context(self, n: int = 10) -> str:
        """
        Get only the N most recent entries.
        Useful for LLM context where token limits require selective history.
        
        Args:
            n: Number of most recent entries to retrieve (default 10)
        """
        # Slice to get last N entries (most recent)
        recent = self.entries[-n:]
        lines = []
        for entry in recent:
            timestamp_str = entry.timestamp.strftime('%H:%M:%S')
            lines.append(f"[{timestamp_str}] {entry.entry_type}: {entry.content}")
        return "\n".join(lines)

We have implemented a basic append-only context system with:
- **ContextEntry dataclass**: Captures timestamp, type, content, and metadata for each entry.
- **AppendOnlyContext class**: Maintains a chronologically ordered list that only grows.
    - The `entries` list is the single source of truth - it is append-only, meaning `list.append()` is the only mutation operation.
    - Append method: Adds new entries without ever modifying existing ones.
    - Chronological ordering is maintained automatically since entries are added sequentially with timestamps.
- **Retrieval methods**: Get full context or recent N entries.
    - The `get_recent_context()` uses Python's negative indexing (`[-n:]`) to efficiently get the last N items without copying the entire list.

Let's test the append-only strategy by simulating a conversation where a user changes their budget midway through. This is a perfect scenario to demonstrate the power of append-only context - when the user initially says their budget is 2000, both values remain in the history. An agent can see the entire evolution and understand that the user modified their requirements. This is invaluable for customer service scenarios where knowing the customer's journey helps provide better assistance, and for debugging situations where we need to understand why the agent made certain recommendations based on what information it had at each point in the conversation.

In [5]:
# Create an append-only context for conversation tracking
print("Append-Only Strategy Example: Conversation Tracking")
print("="*60)

# Initialize the context manager - starts with empty entries list
context = AppendOnlyContext()

# Simulate a conversation where the user changes their mind
# Notice how we NEVER modify existing entries, only add new ones
context.append("user_message", "I'm looking for a laptop")
context.append("agent_message", "I can help you find a laptop. What's your budget?")
context.append("user_message", "Around $1500")
context.append("agent_message", "Great! I recommend the Laptop Pro X1 at $1299.")
context.append("user_message", "Actually, I changed my mind. My budget is $2000 now.")
context.append("agent_message", "Excellent! With $2000, I can recommend the Premium X2.")

# Display the complete conversation history
print("\nFull conversation history:\n")
print(context.get_full_context())

# Show entry count
print(f"\nTotal entries: {len(context.entries)}")

Append-Only Strategy Example: Conversation Tracking

Full conversation history:

[14:27:00] user_message: I'm looking for a laptop
[14:27:00] agent_message: I can help you find a laptop. What's your budget?
[14:27:00] user_message: Around $1500
[14:27:00] agent_message: Great! I recommend the Laptop Pro X1 at $1299.
[14:27:00] user_message: Actually, I changed my mind. My budget is $2000 now.
[14:27:00] agent_message: Excellent! With $2000, I can recommend the Premium X2.

Total entries: 6


This pattern is similar to event sourcing in distributed systems where the event log is immutable. Memory grows linearly with O(n) where n is the number of entries - no automatic cleanup. For production, we would add: persistence (write to disk/database), pagination for large histories and optional entry expiration policies.

**Benefits:**
- Complete history preserved: Every interaction is recorded, nothing is lost.
- Budget changed from $1500 to $2000: Both values are visible in history.
- Temporal ordering maintained: Easy to see the sequence of events.
- Simple implementation: Just append, no complex update logic.
- Audit-friendly: Perfect for compliance and debugging.

**Trade-offs:**
- Context grows indefinitely: Will eventually hit token limits or memory constraints.  
- May contain contradictory information: Agent must determine current state from history.
- Retrieval requires filtering: Need to decide what subset to include in prompts.

### Advanced append-only: Event sourcing pattern
Event sourcing takes append-only to the next level by treating all changes as discrete events in a timeline. Instead of storing "user's cart contains Product X" as state, we store events like "ProductAdded(X)" and "ProductRemoved(Y)". The current state is derived by replaying all events from the beginning. This pattern originated in financial systems where every transaction must be traceable, but it is increasingly used in AI agents for its powerful audit capabilities and ability to reconstruct any historical state.

The beauty of event sourcing is that the event stream becomes the single source of truth. If we want to know the state at any point in time, we replay events up to that moment. If there is a bug in our state-building logic, we fix it and rebuild from the events - the events themselves never change. This makes debugging easier: we can replay events with different logic, add logging or step through exactly what happened. For AI agents handling complex multi-step interactions, this visibility is invaluable.

Let's implement an event-sourced system for a shopping cart scenario where we can track all user actions as discrete events. We will define the types of events that can occur, create an event structure with unique identifiers and timestamps, and build a system that can reconstruct the current state by replaying the event history. This demonstrates how event sourcing provides both complete auditability and the ability to rebuild state from scratch.

First, we define the types of events our system can handle and create a structure to represent each event with all necessary metadata.

In [6]:
class EventType(Enum):
    """Types of events in the system. Each event type represents a distinct action that can occur."""
    PREFERENCE_SET = "preference_set"  # User sets a preference for the first time
    PREFERENCE_CHANGED = "preference_changed"  # User updates an existing preference
    ITEM_ADDED = "item_added"  # Product added to shopping cart
    ITEM_REMOVED = "item_removed"  # Product removed from cart
    ORDER_PLACED = "order_placed"  # User completes purchase

@dataclass
class Event:
    """Event in event-sourced system. Represents a single immutable fact about something that happened."""
    event_id: str  # Unique identifier for this event
    event_type: EventType  # Type of event from EventType enum
    timestamp: datetime  # When this event occurred - critical for temporal ordering
    data: Dict[str, Any]  # Event payload - the actual data associated with this event. Structure varies by event type (e.g., {"budget": 1500} for PREFERENCE_SET)

Now we will build the `EventSourcedContext` class that maintains the event stream and can rebuild current state by replaying events. The key insight is that state is not stored directly - it is derived from the event log. This means we can always reconstruct state at any point in time, change our state-building logic without losing data, and maintain perfect auditability.

In [7]:
class EventSourcedContext:
    """Event-sourced append-only context."""
    
    def __init__(self):
        self.events: List[Event] = []  # The event stream - immutable log of everything that happened
        self.event_counter = 0  # Counter for generating unique event IDs
    
    def add_event(self, event_type: EventType, data: Dict[str, Any]):
        """
        Add event to stream. This is the only way to change system state - by recording events.
        
        Args:
            event_type: Type of event occurring
            data: Event payload with relevant information
        
        Returns:
            The created Event object
        """
        # Increment counter to get next unique ID
        self.event_counter += 1

        # Create event with unique ID, type, current timestamp, and data
        event = Event(
            event_id=f"evt_{self.event_counter:04d}",
            event_type=event_type,
            timestamp=datetime.now(),
            data=data
        )
        # Append to event stream - this is the only mutation
        self.events.append(event)
        return event
    
    def get_current_state(self) -> Dict[str, Any]:
        """
        Rebuild current state from event stream. This demonstrates the power of event sourcing - state is derived, not stored.
        
        Returns:
            Dictionary representing current state
        """
        # Initialize empty state structure
        state = {
            "preferences": {},  # User preferences (budget, category, etc.)
            "cart": [],  # Items currently in shopping cart
            "orders": []  # Completed orders
        }

        # Replay all events in chronological order to build state
        for event in self.events:
            # Handle preference events - merge into preferences dict
            if event.event_type in [EventType.PREFERENCE_SET, EventType.PREFERENCE_CHANGED]:
                state["preferences"].update(event.data)

            # Handle item added to cart
            elif event.event_type == EventType.ITEM_ADDED:
                state["cart"].append(event.data)

            # Handle item removed from cart
            elif event.event_type == EventType.ITEM_REMOVED:
                # Filter out the item with matching product_id
                state["cart"] = [item for item in state["cart"] 
                               if item.get("product_id") != event.data.get("product_id")]

            # Handle order placement - move cart to orders and clear cart
            elif event.event_type == EventType.ORDER_PLACED:
                state["orders"].append(event.data)
                state["cart"] = []   # Cart is emptied after order placement
        
        return state
    
    def get_event_history(self) -> str:
        """
        Get formatted event history. Useful for debugging and understanding what happened.
        
        Returns:
            Human-readable string of all events
        """
        lines = []
        for event in self.events:
            timestamp_str = event.timestamp.strftime('%H:%M:%S')
            # Format: evt_0001 [12:34:56] preference_set: {'budget': 1500}
            lines.append(f"{event.event_id} [{timestamp_str}] {event.event_type.value}: {event.data}")
        return "\n".join(lines)

We have implemented an event-sourced context system with:
- `EventType` enum: Defines all possible event categories in the system.
- `Event` dataclass: Immutable record of what happened with unique ID, type, timestamp, and payload data. Events are immutable - once created, they never change (enforced by using `@dataclass` with no setters).
- `EventSourcedContext` class: Maintains event stream and provides state reconstruction.
    - The event stream (`self.events`) is the single source of truth—all state is derived from it.
- `add_event method`: Only way to change state - by appending new events (never modifying existing ones).
    - Event IDs are monotonically increasing with zero-padding for sortability (`evt_0001`, `evt_0002`, etc.).
- `get_current_state method`: Derives current state by replaying all events from the beginning. State reconstruction uses a fold/reduce pattern: start with empty state, apply each event in sequence.
- `get_event_history method`: Returns formatted event log for audit/debugging.

Let's demonstrate event sourcing with a shopping cart example. We will record various events - setting preferences, adding/removing items, placing orders - and then show how we can view both the complete event history and the current derived state. This illustrates the dual benefit of event sourcing: complete audit trail plus immediate access to current state.

In [8]:
# Example: Shopping cart with event sourcing
print("\nEvent-Sourced Append-Only Example: Shopping Cart")
print("="*60)

# Initialize the EventSourcedContext
cart_context = EventSourcedContext()

# Simulate a sequence of events - each represents a user action
# Event 1: User sets initial preferences
cart_context.add_event(EventType.PREFERENCE_SET, {"budget": 1500, "category": "laptops"})

# Event 2: User adds first item to cart
cart_context.add_event(EventType.ITEM_ADDED, {"product_id": "LP001", "name": "Laptop Pro X1", "price": 1299})

# Event 3: User changes budget preference (doesn't delete old value, adds new event)
cart_context.add_event(EventType.PREFERENCE_CHANGED, {"budget": 2000})

# Event 4: User removes previous item from cart
cart_context.add_event(EventType.ITEM_REMOVED, {"product_id": "LP001"})

# Event 5: User adds higher-end item matching new budget
cart_context.add_event(EventType.ITEM_ADDED, {"product_id": "LP002", "name": "Premium X2", "price": 1899})

# Event 6: User completes purchase
cart_context.add_event(EventType.ORDER_PLACED, {"order_id": "ORD-001", "total": 1899})

# Display the complete event history - shows every action that occurred
print("\nEvent History:\n")
print(cart_context.get_event_history())

# Rebuild current state by replaying all events. This demonstrates that state is derived, not stored
print("\n" + "="*60)
print("\nCurrent State (rebuilt from events):\n")
current_state = cart_context.get_current_state()
print(json.dumps(current_state, indent=2))


Event-Sourced Append-Only Example: Shopping Cart

Event History:

evt_0001 [14:27:03] preference_set: {'budget': 1500, 'category': 'laptops'}
evt_0002 [14:27:03] item_added: {'product_id': 'LP001', 'name': 'Laptop Pro X1', 'price': 1299}
evt_0003 [14:27:03] preference_changed: {'budget': 2000}
evt_0004 [14:27:03] item_removed: {'product_id': 'LP001'}
evt_0005 [14:27:03] item_added: {'product_id': 'LP002', 'name': 'Premium X2', 'price': 1899}
evt_0006 [14:27:03] order_placed: {'order_id': 'ORD-001', 'total': 1899}


Current State (rebuilt from events):

{
  "preferences": {
    "budget": 2000,
    "category": "laptops"
  },
  "cart": [],
  "orders": [
    {
      "order_id": "ORD-001",
      "total": 1899
    }
  ]
}


**Benefits of event sourcing**:
  - Can rebuild state at any point in time by replaying events up to that point.
  - Complete audit trail showing every action that occurred.
  - Can replay events for testing/debugging with different logic.
  - Current state is derived, not stored.
  - Can answer temporal questions: 'What was in cart after event 3?'.
  - If state-building logic has bugs, fix it and rebuild from same events.

For production with millions of events, we would add: snapshots (cached state at intervals), event compaction, and event store (database optimized for append-only workloads like EventStoreDB). Event sourcing is similar to database transaction logs or Git history - complete record of all changes.

## Part 2: Update-in-place strategy

The update-in-place strategy modifies existing context when information changes rather than appending new entries. Think of it as maintaining a whiteboard where we erase old values and write new ones - only the current state is visible. When a user changes their budget from €1500 to €2000, we don't keep both values; we simply replace €1500 with €2000. This keeps context minimal and unambiguous, making it perfect for scenarios where only the current state matters and history provides no value.

Update-in-place is the most token-efficient strategy because context size stays constant regardless of how many updates occur. A shopping cart that gets modified 50 times still occupies the same space whether it has 1 item or 5 items - the number of modifications is irrelevant, only the final state matters. This makes update-in-place ideal for real-time systems, form data, current session state, and anywhere we need to minimize context window consumption. The agent never sees contradictions because old values are gone, replaced by current truth.

The trade-off is permanent information loss. Once we update budget from €1500 to €2000, the original €1500 is gone forever. We can't debug why recommendations changed, can't audit what the user originally wanted, and can't rollback if the update was a mistake. For many use cases this is perfectly acceptable - a form field, a shopping cart, current search filters - but for others it is a critical limitation. Understanding when history doesn't matter is key to using update-in-place effectively.

### Implementing basic update-in-place
Let's build a simple key-value store that maintains current state with update timestamps for each fact. The core idea is straightforward: when we set a value, if it already exists, we simply overwrite it. No history is preserved, no versions are created - just direct replacement. This simplicity makes update-in-place extremely efficient both in terms of memory and computational overhead. The only concession to history we will make is tracking when each fact was last updated, which helps with understanding recency without storing full history.

In [9]:
class UpdateInPlaceContext:
    """Update-in-place context maintenance - overwrites existing values."""
    
    def __init__(self):
        # Dictionary mapping keys to their current values. Old values are discarded when updated
        self.facts: Dict[str, Any] = {}
        
        # Track when each fact was last updated. Useful for knowing recency without storing history
        self.last_updated: Dict[str, datetime] = {}
    
    def set(self, key: str, value: Any):
        """Set or update a fact (overwrites if exists)."""
        # Simply overwrite - old value is lost permanently
        self.facts[key] = value
        
        # Update the timestamp for this key
        self.last_updated[key] = datetime.now()
    
    def get(self, key: str) -> Optional[Any]:
        """Get current value of a fact."""
        # Return current value or None if key doesn't exist
        return self.facts.get(key)
    
    def delete(self, key: str):
        """Remove a fact completely."""
        if key in self.facts:
            # Delete from both dictionaries
            del self.facts[key]
            del self.last_updated[key]
    
    def get_context(self) -> str:
        """Get current context as formatted string."""
        if not self.facts:
            return "No context available."
        
        lines = ["Current context:"]
        # Show each fact with its last update time
        for key, value in self.facts.items():
            updated = self.last_updated[key].strftime('%H:%M:%S')
            lines.append(f"  {key}: {value} (updated: {updated})")
        return "\n".join(lines)

We have implemented a minimal update-in-place context system with:
- **Two dictionaries**: One for current values (`facts`), one for timestamps (`last_updated`).
    - The `last_updated` tracking provides minimal temporal awareness without storing history.
- **Set method**: Directly overwrites existing values with new ones - no history preservation.
- **Get method**: Returns current value only.
- **Delete method**: Removes facts completely from both dictionaries.

Let's test the update-in-place strategy by setting initial preferences and then updating them to see how old values are replaced.

In [10]:
# Create an update-in-place context for user preferences
print("Update-in-Place Strategy Example: User Preferences")
print("="*60)

preferences = UpdateInPlaceContext()

# Set initial preferences
print("\nSetting initial preferences...")
preferences.set("category", "laptops")
preferences.set("brand_preference", "Dell")

print(preferences.get_context())

# Update some preferences - old values will be lost
print("\n" + "-"*60)
print("\nUpdating budget to $2000...")
preferences.set("budget", 2000)  # $1500 is now gone forever

print("\nChanging brand preference to HP...")
preferences.set("brand_preference", "HP")  # "Dell" is now gone forever

print("\n" + preferences.get_context())

Update-in-Place Strategy Example: User Preferences

Setting initial preferences...
Current context:
  category: laptops (updated: 14:27:05)
  brand_preference: Dell (updated: 14:27:05)

------------------------------------------------------------

Updating budget to $2000...

Changing brand preference to HP...

Current context:
  category: laptops (updated: 14:27:05)
  brand_preference: HP (updated: 14:27:05)
  budget: 2000 (updated: 14:27:05)


No need for version management, conflict resolution, or consolidation—just simple key-value updates. This pattern is similar to Redis or Memcached where only current state exists. In production, we would add: persistence to database, atomic updates for concurrent access and optional change notifications.

**Benefits:**
- Minimal context size: Only current values, no history overhead.
- No contradictory information: Agent always sees current truth.
- Easy to query current state: Simple dictionary lookup.
- Token efficient: Perfect for staying within context limits.
- Very simple: No complex logic, easy to understand and maintain.

**Trade-offs:**
- No history of changes: Can't see budget was originally $1500.
- Can't audit what changed when: Lost information cannot be recovered.
- No rollback capability: Updates are permanent and irreversible.

### Advanced update-in-place: Structured state management
While simple key-value storage works for flat data, real applications often need structured state with validation. Using Pydantic models for update-in-place provides type safety, nested structure management, and clear schemas that both humans and AI agents can understand. Instead of raw dictionaries, we define explicit data models that represent the shape of our state, ensuring updates maintain consistency and validity throughout the application lifecycle. The model acts as both documentation and validation - we can see exactly what fields exist, what types they should be and what constraints apply.

Pydantic models also give us powerful features like optional fields with defaults, nested dictionaries and lists with proper typing, automatic validation when values are set, and easy serialization to JSON for persistence or API communication. For AI agent applications, this structured approach is particularly valuable because the schema itself can be included in the agent's context, helping it understand what information is available and how to update it correctly. Let's build a user profile management system using Pydantic that demonstrates these benefits while maintaining the efficiency of update-in-place.

First, let's define the Pydantic model that represents our user profile structure.

In [11]:
class UserProfile(BaseModel):
    """Structured user profile with Pydantic validation. Defines the schema for all user-related state."""
    # Optional fields default to None - allows gradual profile building
    # User can start shopping without providing name/email immediately
    name: Optional[str] = None
    email: Optional[str] = None
    
    # Dictionary for flexible preferences - any key-value pairs. Allows dynamic addition of preferences without schema changes
    preferences: Dict[str, Any] = Field(default_factory=dict)
    
    # List for cart items - maintains insertion order. Each item is a dict with product_id, name, price, etc.
    cart: List[Dict[str, Any]] = Field(default_factory=list)

Now let's build the context manager that uses this Pydantic model and provides convenient methods for updating different parts of the profile.

In [12]:
class StructuredUpdateContext:
    """Update-in-place with structured Pydantic state."""
    
    def __init__(self):
        # Single Pydantic model holds all state. Updates modify this model in-place - no history preserved
        self.profile = UserProfile()
        
        # Track when profile was last updated. Only timestamp, not full history - minimal metadata
        self.updated_at = datetime.now()
    
    def update_preference(self, key: str, value: Any):
        """
        Update a specific preference in the nested dictionary.
        
        Args:
            key: Preference key (e.g., "budget", "category")
            value: New value to set (old value is discarded)
        """
        # Modify the preferences dict within the Pydantic model. This is direct mutation - old value is lost
        self.profile.preferences[key] = value
        self.updated_at = datetime.now()
    
    def add_to_cart(self, item: Dict[str, Any]):
        """
        Add item to cart list.
        
        Args:
            item: Dict with product info (product_id, name, price, etc.)
        """
        # Append to the cart list - maintains shopping order
        self.profile.cart.append(item)
        self.updated_at = datetime.now()
    
    def remove_from_cart(self, product_id: str):
        """
        Remove item from cart by product ID.
        
        Args:
            product_id: ID of product to remove
        """
        # Filter out the item with matching product_id
        # Creates new list without the removed item
        self.profile.cart = [item for item in self.profile.cart 
                            if item.get("product_id") != product_id]
        self.updated_at = datetime.now()
    
    def update_user_info(self, **kwargs):
        """
        Update user information fields (name, email, etc.).
        
        Args:
            **kwargs: Field names and values to update
        """
        # Dynamically set attributes on the Pydantic model. Only sets fields that actually exist in the schema
        for key, value in kwargs.items():
            if hasattr(self.profile, key):
                setattr(self.profile, key, value)
        self.updated_at = datetime.now()
    
    def get_context_for_agent(self) -> str:
        """
        Get formatted context optimized for agent consumption. Only includes non-None/non-empty fields for efficiency.
        
        Returns:
            Human-readable string representation of current state
        """
        lines = ["User Profile:"]
        
        # Only include fields that have been set (not None). Keeps context minimal - agent doesn't see unset fields
        if self.profile.name:
            lines.append(f"  Name: {self.profile.name}")
        if self.profile.email:
            lines.append(f"  Email: {self.profile.email}")
        
        # Show preferences if any exist
        if self.profile.preferences:
            lines.append("\nPreferences:")
            for key, value in self.profile.preferences.items():
                lines.append(f"  {key}: {value}")
        
        # Show cart items if any exist
        if self.profile.cart:
            lines.append("\nCart:")
            for item in self.profile.cart:
                lines.append(f"  - {item.get('name', 'Unknown')} (${item.get('price', 0)})")
        
        # Include last update timestamp for recency awareness
        lines.append(f"\nLast updated: {self.updated_at.strftime('%Y-%m-%d %H:%M:%S')}")
        
        return "\n".join(lines)

Here, we demonstrate structured update-in-place with Pydantic validation through these operations:
- Initialization: Creates `StructuredUpdateContext` instance with empty `UserProfile` Pydantic model.
- User info updates: Uses `setattr()` to dynamically set name and email fields on the Pydantic model.
- Preference management: Modifies the nested preferences dictionary directly (`self.profile.preferences[key] = value`).
- Cart operations: Appends items to the `cart` list or filters out items using list comprehension.
- State retrieval: `get_context_for_agent()` iterates through non-empty fields to build a formatted string.

Now let's see how this structured update-in-place context works in a realistic e-commerce scenario.

In [13]:
# Example: E-commerce session with structured state
print("\nStructured Update-in-Place Example: E-commerce Session")
print("="*60)

# Create the context manager with empty Pydantic profile
session = StructuredUpdateContext()

# Build user profile through incremental updates
print("\nBuilding user profile...\n")

session.update_user_info(name="Alice", email="alice@example.com") # Set user identification information
# Set shopping preferences
session.update_preference("budget", 1500)  # Initial budget of 1500
session.update_preference("category", "laptops")  # Looking for laptops
session.add_to_cart({"product_id": "LP001", "name": "Laptop Pro X1", "price": 1299})  # Add initial product to cart

print("After initial setup:")
print(session.get_context_for_agent())

print("\n" + "-"*60)
print("\nUser changes mind - updating budget and cart...\n")

# Update preferences and cart - old values replaced
session.update_preference("budget", 2000)  # User decides to increase budget
session.remove_from_cart("LP001")  # Remove the original laptop from cart
session.add_to_cart({"product_id": "LP002", "name": "Premium X2", "price": 1899})  # Add the premium laptop that fits the new budget

print("After updates:")
print(session.get_context_for_agent())


Structured Update-in-Place Example: E-commerce Session

Building user profile...

After initial setup:
User Profile:
  Name: Alice
  Email: alice@example.com

Preferences:
  budget: 1500
  category: laptops

Cart:
  - Laptop Pro X1 ($1299)

Last updated: 2025-12-14 14:27:09

------------------------------------------------------------

User changes mind - updating budget and cart...

After updates:
User Profile:
  Name: Alice
  Email: alice@example.com

Preferences:
  budget: 2000
  category: laptops

Cart:
  - Premium X2 ($1899)

Last updated: 2025-12-14 14:27:09


This approach provides efficient state management with validation but sacrifices auditability - there's no way to recover what the budget was before the update or see that LP001 was ever in the cart.

Benefits of structured updates:
  - Type-safe with Pydantic validation.
  - Clear schema for agent understanding.
  - Efficient storage of current state.
  - Easy to serialize/deserialize (JSON, database).
  - Nested structures (preferences, cart) managed cleanly.

## Part 3: Versioned updates strategy
The versioned updates strategy keeps both current state AND change history by storing multiple versions of each fact. Think of it as Git for our context - every change creates a new version while preserving all previous versions. When a user changes their budget from €1500 to €2000, we don't choose between keeping one or the other; we keep both as version 1 and version 2. This gives us the best of both worlds: quick access to current state plus complete history for debugging, analytics and rollback.

Versioning solves a critical problem that neither append-only nor update-in-place can handle well: we need current state to be immediately accessible (like update-in-place) but we also need to track how it evolved over time (like append-only). With versioned updates, getting the current budget is a single operation - just grab the latest version. But if we need to debug why recommendations changed, we can walk through the version history: v1 was €1000, v2 was €1500, v3 is €2000. Each version can optionally include a reason for the change, creating a narrative of decision-making.

The storage cost is higher than update-in-place but more structured than append-only. Instead of keeping every conversation turn mentioning budget, we keep only the distinct budget values as versions. This is much more efficient than full append-only while providing similar historical reconstruction capabilities. The trade-off is implementation complexity - we need to manage version lists, decide what to include in context (current only? recent history? full history?), and potentially implement time-travel queries or rollback functionality.

### When to use versioned updates:
- User preferences where we need both current values and change history.
- Configuration management where rollback might be needed.
- Debugging scenarios where understanding state evolution is critical.
- Analytics that track how user needs evolve over time.
- Audit requirements that demand versioning but not full append-only logs.

To implement versioned updates, we need a system that tracks each distinct value a fact has held over time, along with metadata about when and why it changed. The foundation is a version object that bundles together the version number, the actual value at that version, a timestamp marking when this version was created, and optionally a human-readable reason explaining why the change occurred. This creates a rich historical record where we can look back and understand not just what changed, but when and why it changed. Then we need a context manager that maintains lists of these versions for each fact, providing methods to add new versions, retrieve the current latest version efficiently, access any historical version, and view the complete evolution of how a fact changed over time. Let's start by defining the structure for a single version.

In [14]:
@dataclass
class FactVersion:
    """A single version of a fact with metadata."""
    # Version number (1, 2, 3, ...) for this fact
    version: int
    
    # The actual value at this version - can be any type (int, str, dict, list, etc.)
    value: Any
    
    # When this version was created - crucial for understanding temporal evolution
    timestamp: datetime
    
    # Optional reason/explanation for why this changed - helps with debugging and understanding evolution
    reason: Optional[str] = None

This dataclass serves as the immutable record of a single version. Each time a fact changes, we create a new `FactVersion` instance rather than modifying existing ones. The version number provides ordering, the timestamp enables temporal queries, and the optional reason field adds human context that makes debugging and analysis much easier. 

Now let's build the context manager that will maintain collections of these versions.

In [15]:
class VersionedContext:
    """Versioned context maintaining change history for each fact."""
    
    def __init__(self):
        # Maps each fact key to a list of all its versions - Versions are stored in chronological order (oldest first, newest last)
        # Example: {"budget": [v1, v2, v3], "category": [v1, v2]}
        self.facts: Dict[str, List[FactVersion]] = {}
    
    def set(self, key: str, value: Any, reason: str = None):
        """
        Set a fact by creating a new version (doesn't delete old versions).
        
        Args:
            key: The fact identifier (e.g., "budget", "category")
            value: The new value for this fact
            reason: Optional explanation for this change
        """
        # Initialize version list for this key if it doesn't exist
        if key not in self.facts:
            self.facts[key] = []
        
        # Version number is the next in sequence
        version = len(self.facts[key]) + 1
        
        # Create new version object with all metadata
        fact_version = FactVersion(
            version=version,
            value=value,
            timestamp=datetime.now(),
            reason=reason
        )
        
        # Append to version history - old versions preserved - This is the only mutation - we never modify or delete existing versions
        self.facts[key].append(fact_version)
    
    def get_current(self, key: str) -> Optional[Any]:
        """
        Get current (latest) value efficiently using list indexing.
        
        Args:
            key: The fact identifier
            
        Returns:
            The latest value or None if key doesn't exist
        """
        # Check if key exists and has versions
        if key in self.facts and self.facts[key]:
            # Return the value from the last version
            return self.facts[key][-1].value
        return None
    
    def get_version(self, key: str, version: int) -> Optional[Any]:
        """
        Get specific version of a fact (1-indexed).
        
        Args:
            key: The fact identifier
            version: Version number
            
        Returns:
            The value at that version or None if invalid
        """
        # Validate version number is in valid range
        if key in self.facts and 0 < version <= len(self.facts[key]):
            # Access by index (version 1 = index 0)
            return self.facts[key][version - 1].value
        return None
    
    def get_history(self, key: str) -> List[FactVersion]:
        """
        Get all versions of a fact for analysis.
        
        Args:
            key: The fact identifier
            
        Returns:
            List of all FactVersion objects for this key
        """
        # Return the complete version list, or empty list if key doesn't exist
        return self.facts.get(key, [])
    
    def get_current_context(self) -> str:
        """
        Get context showing only current state.
        Useful when you want the latest values without history noise.
        
        Returns:
            Formatted string showing current state
        """
        lines = ["Current state:"]
        for key, versions in self.facts.items():
            # Get the latest version from the list
            current = versions[-1]
            # Show the current value with version number for reference
            lines.append(f"  {key}: {current.value} (v{current.version})")
        return "\n".join(lines)
    
    def get_full_context_with_history(self) -> str:
        """
        Get context including complete change history.
        Useful for debugging and understanding how state evolved.
        
        Returns:
            Formatted string showing all versions of all facts
        """
        lines = ["Context with history:"]
        for key, versions in self.facts.items():
            lines.append(f"\n{key}:")
            # Show each version with timestamp and reason
            for v in versions:
                timestamp = v.timestamp.strftime('%H:%M:%S')
                reason_str = f" - {v.reason}" if v.reason else ""
                lines.append(f"  v{v.version} [{timestamp}]: {v.value}{reason_str}")
        return "\n".join(lines)

The `VersionedContext` class provides complete version history management:
- Storage structure: Dictionary mapping each fact key to a chronologically-ordered list of `FactVersion` objects.
- Set operation: Appends new versions without touching existing ones - implements true immutability.
- Current value retrieval: O(1) access via negative list indexing (`[-1]` gets last element).
- Historical access: Can retrieve any specific version by number (1-indexed for human readability).
- History analysis: Returns complete version lists for understanding evolution patterns.
- Dual context views: Can format output as either current-only or full-history depending on needs.

Now let's see this in action by tracking how a user's shopping preferences evolve over time, complete with explanatory reasons for each change.

In [16]:
# Example: Tracking preference changes with reasons
print("Versioned Updates Strategy Example: Preference Tracking")
print("="*60)

versioned_ctx = VersionedContext()

# Track changes over time with explanatory reasons
print("\nTracking budget changes...\n")

versioned_ctx.set("budget", 1000, "Initial budget")  # Set initial budget - creates version 1
versioned_ctx.set("category", "laptops", "Shopping for laptops")  # Set initial category - creates version 1 for category
versioned_ctx.set("budget", 1500, "Found extra money")  # Update budget - creates version 2 (version 1 still preserved)
versioned_ctx.set("budget", 2000, "Decided to invest more")  # Update budget again - creates version 3 (versions 1 and 2 still preserved)
versioned_ctx.set("category", "gaming_laptops", "Changed to gaming focus")  # Update category - creates version 2 for category

# Show current state (quick access to latest values)
print("Current state:")
print(versioned_ctx.get_current_context())

# Show full history (for debugging/analytics) - displays all versions chronologically
print("\n" + "="*60)
print("\nFull history:")
print(versioned_ctx.get_full_context_with_history())

# Query specific versions and evolution
print("\n" + "="*60)
print("\nQuerying history:")
print(f"Original budget: ${versioned_ctx.get_version('budget', 1)}")  # Get specific historical version (1-indexed)
print(f"Current budget: ${versioned_ctx.get_current('budget')}")  # Get current version (latest value)

# Analyze complete evolution of a single fact
print("\nBudget evolution:")
for v in versioned_ctx.get_history('budget'):
    print(f"  v{v.version}: ${v.value} - {v.reason}")

Versioned Updates Strategy Example: Preference Tracking

Tracking budget changes...

Current state:
Current state:
  budget: 2000 (v3)
  category: gaming_laptops (v2)


Full history:
Context with history:

budget:
  v1 [14:27:13]: 1000 - Initial budget
  v2 [14:27:13]: 1500 - Found extra money
  v3 [14:27:13]: 2000 - Decided to invest more

category:
  v1 [14:27:13]: laptops - Shopping for laptops
  v2 [14:27:13]: gaming_laptops - Changed to gaming focus


Querying history:
Original budget: $1000
Current budget: $2000

Budget evolution:
  v1: $1000 - Initial budget
  v2: $1500 - Found extra money
  v3: $2000 - Decided to invest more


- Budget has 3 versions (1000 → 1500 → 2000), all preserved and queryable.
- Category has 2 versions (laptops → gaming_laptops), both accessible.
- Current state shows only latest values but includes version numbers for reference.
- Full history reveals the complete evolution with timestamps and reasons.
- Can answer questions like "what was the original budget?" or "how many times did budget change?".

**Benefits of versioned updates**:
- Current state readily available with O(1) access to latest version.
- Complete change history preserved for audit and debugging.
- Can track why changes happened via reason field.
- Can rollback to any version or analyze evolution patterns.
- More structured than append-only (versions grouped by fact).
- Great for analytics showing how preferences evolved.

**Drawbacks**:
- More storage than update-in-place (grows with number of changes).
- Need to decide what to include in agent context (current only? recent versions? full history?).
- Complexity in managing version lists.
- Must prevent unbounded growth in production (version limits or archival strategies).

### Advanced versioning: Time-travel queries
Time-travel queries extend versioning to answer temporal questions: "What was the state at 2pm yesterday?" or "What changed between Monday and Wednesday?" This powerful capability enables debugging ("what was the budget when we made that recommendation?"), compliance ("show me the state during the audit period"), and analytics ("how did preferences evolve during the marketing campaign?"). By combining version history with timestamp filtering, we can reconstruct any historical state or analyze change patterns over time.

Building on top of versioned updates, we can implement powerful time-travel capabilities that let us reconstruct the exact state of the context at any moment in the past, or analyze all changes that occurred within a specific time window. This is incredibly valuable for debugging scenarios where we need to understand what the agent "knew" at the moment it made a particular decision, or for compliance audits that require showing the state during a specific period. The key insight is that by walking through our version history and filtering by timestamps, we can answer temporal questions without needing separate snapshots or backups. Every historical state is implicitly available in the version log, we just need the right query methods to extract it. Let's extend our `VersionedContext` class with time-travel methods.

In [17]:
class TimeTravelContext(VersionedContext):
    """Versioned context with time-travel query capabilities."""
    
    def get_state_at_time(self, timestamp: datetime) -> Dict[str, Any]:
        """
        Reconstruct state as it existed at a specific moment in time. This is like asking: "What did the context look like at 2pm yesterday?"
        
        Args:
            timestamp: The point in time to query
            
        Returns:
            Dictionary of fact values as they existed at that time
        """
        state = {}
        
        # For each fact, find its value at the specified time
        for key, versions in self.facts.items():
            # Walk backwards through versions to find latest before timestamp
            for version in reversed(versions):
                if version.timestamp <= timestamp:
                    # This is the most recent version at or before the timestamp
                    state[key] = version.value
                    break  # Found it, move to next fact
        
        return state
    
    def get_changes_between(self, start: datetime, end: datetime) -> Dict[str, List[FactVersion]]:
        """
        Get all changes that occurred in a time window. This is like asking: "What changed between Monday and Wednesday?"
        
        Args:
            start: Beginning of time window
            end: End of time window
            
        Returns:
            Dictionary mapping fact keys to their versions within the window
        """
        changes = {}
        
        # For each fact, find versions within the time window
        for key, versions in self.facts.items():
            # Filter to versions in the time range
            key_changes = [v for v in versions if start <= v.timestamp <= end]
            
            # Only include facts that actually changed during this window
            if key_changes:
                changes[key] = key_changes
        
        return changes
    
    def compare_versions(self, key: str, version1: int, version2: int) -> Dict[str, Any]:
        """
        Compare two specific versions to see what changed. Useful for understanding specific transitions.
        
        Args:
            key: The fact to compare
            version1: First version number
            version2: Second version number
            
        Returns:
            Dictionary with comparison results
        """
        v1 = self.get_version(key, version1)
        v2 = self.get_version(key, version2)
        
        return {
            "key": key,
            "version1": {"version": version1, "value": v1},
            "version2": {"version": version2, "value": v2},
            "changed": v1 != v2  # Did the value actually change?
        }

The `TimeTravelContext` extends `VersionedContext` with three powerful temporal query methods:
- Point-in-time reconstruction: `get_state_at_time()` walks backward through version history, finding the most recent version before target timestamp for each fact.
- Time-window analysis: `get_changes_between()` filters versions by timestamp range to show what changed during a period.
- Version comparison: `compare_versions()` directly compares two specific versions to understand transitions.

These methods enable debugging workflows like "show me the context when the agent made recommendation X" and compliance workflows like "prove what data we had during the audit period". Let's demonstrate with a realistic example using measurable time gaps.

In [18]:
# Example: Time-travel debugging with checkpoints
print("\nTime-Travel Context Example")
print("="*60)

tt_ctx = TimeTravelContext()

# Record changes with measurable time gaps
start_time = datetime.now()  # Record initial state
print(f"\nStart time: {start_time.strftime('%H:%M:%S.%f')}")
tt_ctx.set("budget", 1000, "Initial")
tt_ctx.set("category", "laptops", "Initial")

# Sleep to create time separation for demonstration
time.sleep(0.1)
checkpoint1 = datetime.now()
tt_ctx.set("budget", 1500, "Increased")

# Another time gap
time.sleep(0.1)
checkpoint2 = datetime.now()
tt_ctx.set("budget", 2000, "Increased again")
tt_ctx.set("category", "gaming_laptops", "Changed focus")

end_time = datetime.now()
print(f"End time: {end_time.strftime('%H:%M:%S.%f')}")

# Query state at checkpoint 1 - reconstructs what the context looked like at that moment
print("\nTime-travel query: State at checkpoint 1")
state_at_cp1 = tt_ctx.get_state_at_time(checkpoint1)
print(json.dumps(state_at_cp1, indent=2))

# Query state at checkpoint 2 - shows later state after more changes
print("\nTime-travel query: State at checkpoint 2")
state_at_cp2 = tt_ctx.get_state_at_time(checkpoint2)
print(json.dumps(state_at_cp2, indent=2))

# Analyze what changed in a time window - shows all versions created during the period
print("\nChanges between checkpoint 1 and 2:")
changes = tt_ctx.get_changes_between(checkpoint1, checkpoint2)
for key, versions in changes.items():
    print(f"\n{key}:")
    for v in versions:
        print(f"  v{v.version}: {v.value} - {v.reason}")


Time-Travel Context Example

Start time: 14:27:15.584725
End time: 14:27:15.785570

Time-travel query: State at checkpoint 1
{
  "budget": 1000,
  "category": "laptops"
}

Time-travel query: State at checkpoint 2
{
  "budget": 1500,
  "category": "laptops"
}

Changes between checkpoint 1 and 2:

budget:
  v2: 1500 - Increased


The demonstration shows three types of temporal queries working together:
- Point-in-time reconstruction: `get_state_at_time(checkpoint1)` iterates through all facts, walking backward through each fact's version list until finding a version with `timestamp <= checkpoint1`.
- State evolution: Checkpoint 1 captured state when budget was 1500 and category was still "laptops", while checkpoint 2 captured state after budget reached 2000 and category changed to "gaming_laptops".
- Time-window filtering: `get_changes_between()` uses list comprehension with timestamp comparison to isolate versions created within the specified range.

Practical applications observed:
- Debugging: "Show me the exact context when the agent recommended the $1299 laptop" → reconstruct state at that moment.
- Compliance: "Prove what user data we had on date X" → get_state_at_time for that date.
- Analytics: "What changed during our A/B test period?" → get_changes_between test start and end dates.
- Rollback: "Restore context to 5 minutes ago" → get_state_at_time for 5 minutes ago, then rebuild.

Benefits of time-travel:
- Reconstruct any historical state without storing explicit snapshots.
- Understand exactly what the agent "knew" at any moment.
- Analyze change patterns within specific time periods.
- Support compliance and audit requirements with temporal proof.
- Enable rollback to any point in time for recovery scenarios.

This temporal query capability transforms versioned storage from simple history tracking into a powerful debugging and compliance tool.

## Part 4: Structured merging strategy

The structured merging strategy intelligently combines new information with existing context using configurable conflict resolution rules. Unlike the previous strategies that either keep everything (append-only), keep nothing (update-in-place), or keep versions (versioned), structured merging makes intelligent decisions about how to handle conflicts when new data arrives. Should we keep the old value, use the new one, combine both or apply custom logic? The answer depends on context, and structured merging provides the framework to express these rules.

Merging becomes critical when context comes from multiple sources with different reliability levels. Imagine an e-commerce agent receiving budget information from three sources: user explicit statement (€1500), browsing history (€1000), and saved preferences (€1200). Which is correct? With priority-based merging, we assign each source a priority - user input might be priority 10, saved preferences priority 7, browsing history priority 5 - and the system automatically keeps the highest-priority value. This prevents low-quality data from overwriting high-quality data.

Beyond priority, structured merging offers multiple strategies for different data types. Lists might use APPEND to combine values from multiple sources. Dictionaries might use MERGE_DEEP to recursively combine nested structures. Timestamps can drive TAKE_NEWER logic. Each field can have its own merge strategy and provenance tracking shows exactly where each piece of data came from. This level of control is essential for production systems where data quality and consistency are critical.

### When to use structured merging:
- Combining information from multiple data sources (user input, databases, APIs, inference).
- Updating complex nested structures (user profiles, configurations).
- Resolving conflicts with explicit policies (priority, recency, custom logic).
- Maintaining data provenance (tracking where each field came from).
- Systems requiring conflict resolution guarantees.

To build a structured merging system, we first need to define the different strategies for handling conflicts when new data arrives. Different types of data require different merge behaviors - sometimes we want to keep the existing value, sometimes we want to overwrite it, sometimes we want to combine both values, and sometimes we need to make intelligent decisions based on metadata like timestamps or source priority. By defining these strategies as an enumeration, we create a clear vocabulary for expressing merge policies that can be applied consistently across different data types and sources. Let's start by defining the available merge strategies.

In [19]:
class MergeStrategy(Enum):
    """Strategies for merging conflicting values."""
    KEEP_EXISTING = "keep_existing"  # Keep old value, ignore new value (useful for immutable facts)
    OVERWRITE = "overwrite"  # Replace old value with new value (simple last-write-wins)
    APPEND = "append"  # Combine both values into a list (useful for accumulating multiple inputs)
    MERGE_DEEP = "merge_deep"  # Deep merge dictionaries recursively (useful for nested configuration)
    TAKE_NEWER = "take_newer"  # Use the value with the newer timestamp (recency-based decision)
    TAKE_HIGHER_PRIORITY = "take_higher_priority"  # Use the value from the higher priority source (trust-based decision)

We have defined six distinct merge strategies that cover different conflict resolution scenarios. Each strategy represents a different policy for deciding what to do when new data conflicts with existing data. The choice of strategy depends on the semantics of the data being merged - user preferences might use TAKE_HIGHER_PRIORITY to trust explicit user input over inferred preferences, while conversation topics might use APPEND to accumulate all mentioned topics. Now let's build the context manager that applies these strategies when merging updates.

In [20]:
class StructuredMergeContext:
    """Context with intelligent merging capabilities based on configurable strategies."""
    
    def __init__(self):
        self.data: Dict[str, Any] = {}  # Main data storage - dictionary of current values
        self.metadata: Dict[str, Dict] = {}  # Metadata storage - tracks source, timestamp, priority, and strategy for each field
    
    def merge(self, updates: Dict[str, Any], 
             strategy: MergeStrategy = MergeStrategy.OVERWRITE,
             source: str = "unknown",
             priority: int = 1):
        """
        Merge updates into context using specified strategy.
        
        Args:
            updates: Dictionary of field updates to merge
            strategy: How to handle conflicts (default: OVERWRITE)
            source: Where this data came from (e.g., "user_input", "database", "api")
            priority: Importance level of this source (higher = more trusted)
        """
        # Process each field in the updates dictionary
        for key, new_value in updates.items():
            self._merge_field(key, new_value, strategy, source, priority)
    
    def _merge_field(self, key: str, new_value: Any, 
                    strategy: MergeStrategy, source: str, priority: int):
        """
        Merge a single field using the specified strategy. This is where the actual conflict resolution logic lives.
        
        Args:
            key: Field name
            new_value: New value to merge
            strategy: Merge strategy to apply
            source: Data source identifier
            priority: Source priority level
        """
        # Get existing value and metadata for this field
        existing_value = self.data.get(key)
        existing_meta = self.metadata.get(key, {})
        
        # Apply the appropriate merge strategy
        if strategy == MergeStrategy.KEEP_EXISTING and existing_value is not None:
            # Keep old value, discard new value (unless field doesn't exist yet)
            final_value = existing_value
        
        elif strategy == MergeStrategy.OVERWRITE:
            # Simple replacement - new value overwrites old value
            final_value = new_value
        
        elif strategy == MergeStrategy.APPEND:
            # Accumulate values into a list
            if existing_value is None:
                # No existing value - create list with new value
                final_value = [new_value] if not isinstance(new_value, list) else new_value
            elif isinstance(existing_value, list):
                # Existing value is already a list - append to it
                final_value = existing_value + ([new_value] if not isinstance(new_value, list) else new_value)
            else:
                # Existing value is scalar - create list with both values
                final_value = [existing_value, new_value]
        
        elif strategy == MergeStrategy.MERGE_DEEP:
            # Deep merge dictionaries (shallow merge as simplified version)
            if isinstance(existing_value, dict) and isinstance(new_value, dict):
                # Both are dicts - merge them (new values override existing)
                final_value = {**existing_value, **new_value}
            else:
                # Not both dicts - fall back to overwrite
                final_value = new_value
        
        elif strategy == MergeStrategy.TAKE_NEWER:
            # Compare timestamps - keep the newer value
            existing_time = existing_meta.get('timestamp', datetime.min)
            new_time = datetime.now()
            final_value = new_value if new_time > existing_time else existing_value
        
        elif strategy == MergeStrategy.TAKE_HIGHER_PRIORITY:
            # Compare priorities - keep the higher priority value
            existing_priority = existing_meta.get('priority', 0)
            final_value = new_value if priority >= existing_priority else existing_value
        
        else:
            # Unknown strategy - default to overwrite
            final_value = new_value
        
        # Update data and metadata with results
        self.data[key] = final_value
        self.metadata[key] = {
            'source': source,
            'timestamp': datetime.now(),
            'priority': priority,
            'strategy_used': strategy.value
        }
    
    def get_context(self) -> str:
        """
        Get formatted context showing current values with provenance.
        
        Returns:
            Human-readable string with values and metadata
        """
        lines = ["Current context:"]
        for key, value in self.data.items():
            meta = self.metadata[key]
            # Show value with source and priority for transparency
            lines.append(f"  {key}: {value}")
            lines.append(f"    (source: {meta['source']}, priority: {meta['priority']})")
        return "\n".join(lines)

The `StructuredMergeContext` provides intelligent data merging with full provenance tracking:
- Dual storage: Separate dictionaries for data values and metadata (source, timestamp, priority, strategy used).
- Strategy dispatch: `_merge_field()` implements all six merge strategies with clear conditional logic.
- Provenance tracking: Every field records where it came from and when, enabling audit trails.
- Priority-based conflicts: Can assign different trust levels to different sources (user input vs inference vs defaults).
- Flexible merging: Lists can accumulate, dicts can deep-merge, values can be compared by recency or priority.

Now let's demonstrate these strategies with a practical multi-source data integration scenario where information arrives from different sources with different reliability levels.

In [21]:
# Example: Merging from multiple sources
print("Structured Merging Strategy Example")
print("="*60)

merge_ctx = StructuredMergeContext()

# Step 1: Initial data from user input (highest priority)
print("\n1. Initial data from user:")
merge_ctx.merge(
    {"budget": 1500, "category": "laptops"},
    strategy=MergeStrategy.OVERWRITE,
    source="user_input",
    priority=10  # User input gets highest priority
)
print(merge_ctx.get_context())

# Step 2: Merge browsing history (lower priority - should not overwrite user input)
print("\n" + "-"*60)
print("\n2. Merge browsing history (lower priority):")
merge_ctx.merge(
    {"budget": 1000, "interests": ["gaming", "productivity"]},  # Budget conflicts with user input
    strategy=MergeStrategy.TAKE_HIGHER_PRIORITY,  # Use priority to resolve conflict
    source="browsing_history",
    priority=5  # Lower than user_input priority of 10
)
print(merge_ctx.get_context())
print("\n→ Budget unchanged (user input has higher priority)")

# Step 3: Append additional interest from conversation (accumulate values)
print("\n" + "-"*60)
print("\n3. Append interests from conversation:")
merge_ctx.merge(
    {"interests": "video_editing"},  # Single value to append to existing list
    strategy=MergeStrategy.APPEND,  # Accumulate into list
    source="conversation",
    priority=8
)
print(merge_ctx.get_context())

# Step 4: Deep merge detailed preferences (combine nested dictionaries)
print("\n" + "-"*60)
print("\n4. Deep merge detailed preferences:")

# First set of detailed preferences
merge_ctx.merge(
    {"preferences": {"brand": "Dell", "color": "silver"}},
    strategy=MergeStrategy.MERGE_DEEP,  # Merge dictionaries
    source="user_input",
    priority=10
)

# Second set of detailed preferences (should merge with first, not replace)
merge_ctx.merge(
    {"preferences": {"ram": "16GB", "storage": "512GB"}},
    strategy=MergeStrategy.MERGE_DEEP,  # Merge again
    source="user_input",
    priority=10
)
print(merge_ctx.get_context())
print("\n→ Preferences dictionary now contains all four fields: brand, color, ram, storage")

Structured Merging Strategy Example

1. Initial data from user:
Current context:
  budget: 1500
    (source: user_input, priority: 10)
  category: laptops
    (source: user_input, priority: 10)

------------------------------------------------------------

2. Merge browsing history (lower priority):
Current context:
  budget: 1500
    (source: browsing_history, priority: 5)
  category: laptops
    (source: user_input, priority: 10)
  interests: ['gaming', 'productivity']
    (source: browsing_history, priority: 5)

→ Budget unchanged (user input has higher priority)

------------------------------------------------------------

3. Append interests from conversation:
Current context:
  budget: 1500
    (source: browsing_history, priority: 5)
  category: laptops
    (source: user_input, priority: 10)
  interests: ['gaming', 'productivity', 'video_editing']
    (source: conversation, priority: 8)

------------------------------------------------------------

4. Deep merge detailed prefere

The example demonstrates four different merge strategies working together on multi-source data:
- Priority-based conflict resolution: Budget value of 1000 from browsing history (priority 5) was rejected in favor of 1500 from user input (priority 10) using `TAKE_HIGHER_PRIORITY` strategy.
- List accumulation: Interests field started with list `["gaming", "productivity"]` and accumulated `"video_editing"` via `APPEND` strategy, resulting in three-element list.
- Dictionary merging: Preferences field deep-merged two separate dictionaries `{"brand": "Dell", "color": "silver"}` and `{"ram": "16GB", "storage": "512GB"}` using `MERGE_DEEP` strategy, producing combined dictionary with four keys.
- Metadata tracking: Every field maintains provenance information showing source, timestamp, priority, and strategy used.

Key observations:
- Different fields can use different merge strategies based on their semantics.
- Priority system prevents low-quality data from overwriting high-quality data.
- APPEND enables accumulation from multiple sources without loss.
- MERGE_DEEP allows building complex nested structures incrementally.
- Metadata provides transparency about where each value came from and why it won conflicts.

Benefits of structured merging:
- Intelligently resolves conflicts based on configurable policies.
- Preserves high-priority information when conflicts occur.
- Combines complementary data from multiple sources.
- Maintains consistency through clear merge rules.
- Tracks provenance of each field for audit and debugging.

This approach is essential for production systems that integrate data from user input, databases, APIs, and inference engines, each with different reliability levels.

### Advanced merging: Conflict resolution with LLM
While rule-based merge strategies work well for structured conflicts where priority or recency can make the decision, some conflicts require semantic understanding to resolve properly. When a user says "around 1500 initially and later says "1500-2000 range", these aren't simple contradictions - they are complementary pieces of information that a human would naturally understand as refining rather than replacing. An LLM can understand this nuance and intelligently merge the values into something like "€1500-2000 budget range" that captures both pieces of information. This LLM-assisted merging is particularly valuable for natural language data where strict rule-based strategies would either lose information or create nonsensical combinations. Let's extend our structured merge context with LLM assistance for handling semantic conflicts.

In [22]:
class LLMAssistedMergeContext(StructuredMergeContext):
    """Context merging with LLM assistance for complex semantic conflicts."""
    
    def __init__(self, llm):
        super().__init__()
        self.llm = llm  # LLM instance for semantic conflict resolution
    
    def smart_merge(self, key: str, old_value: Any, new_value: Any, 
                   context: str = "") -> Any:
        """
        Use LLM to resolve conflicting values intelligently. This handles semantic conflicts that rule-based strategies can't solve.
        
        Args:
            key: The field name being merged
            old_value: Existing value
            new_value: New value to merge
            context: Additional context to help LLM make decision
            
        Returns:
            Resolved value (might be old, new, or combination)
        """
        # If values are identical, no conflict to resolve
        if old_value == new_value:
            return new_value

        # Construct prompt asking LLM to resolve the conflict
        prompt = f"""Resolve the conflict between two values for '{key}':

Old value: {old_value}
New value: {new_value}

Context: {context}

Determine the most appropriate value. Consider:
1. Are these contradictory or complementary?
2. Should they be combined or should one replace the other?
3. What makes the most sense given the context?

Return only the resolved value (or a combined value if appropriate).

Resolved value:"""

        # Call LLM to get resolution
        response = self.llm.invoke(prompt)
        return response.content.strip()
    
    def intelligent_merge(self, updates: Dict[str, Any], 
                         source: str = "unknown",
                         priority: int = 1,
                         context: str = ""):
        """
        Merge with LLM assistance for conflicts. Uses LLM to resolve semantic conflicts instead of rule-based strategies.
        
        Args:
            updates: Dictionary of updates to merge
            source: Source identifier
            priority: Source priority
            context: Additional context for LLM decision making
        """
        for key, new_value in updates.items():
            if key in self.data:
                # Conflict detected - use LLM to resolve
                resolved = self.smart_merge(key, self.data[key], new_value, context)
                self.data[key] = resolved
            else:
                # No conflict - just set the value
                self.data[key] = new_value

            # Update metadata
            self.metadata[key] = {
                'source': source,
                'timestamp': datetime.now(),
                'priority': priority,
                'strategy_used': 'llm_assisted'
            }

The `LLMAssistedMergeContext` adds semantic conflict resolution on top of rule-based merging:
- Semantic understanding: LLM analyzes whether values are contradictory or complementary.
- Contextual resolution: Additional context helps LLM make informed decisions about how to merge.
- Flexible combining: Can intelligently combine values like "1500" and "1500-2000 range" into unified representation.
- Prompt-based resolution: Uses structured prompt asking LLM to consider contradiction vs complement and return resolved value.

This is particularly valuable for natural language user inputs where strict rules would fail. Let's see it in action.

In [23]:
# Example: LLM-assisted conflict resolution
print("\nLLM-Assisted Merging Example")
print("="*60)

llm_merge_ctx = LLMAssistedMergeContext(llm)

# Initial preferences - Set initial budget value from user
llm_merge_ctx.merge(
    {"budget": "around $1500"},
    strategy=MergeStrategy.OVERWRITE,
    source="user_message",
    priority=10
)

print("\nInitial state:")
print(f"Budget: {llm_merge_ctx.data['budget']}")

# User later provides slightly different budget information - This creates a semantic conflict that requires understanding, not just rules
print("\n" + "-"*60)
print("\nUser later says something that might conflict...")
print("Using LLM to resolve: 'around $1500' vs '$1500-2000 range'")

# Use LLM to resolve the conflict
llm_merge_ctx.intelligent_merge(
    {"budget": "$1500-2000 range"},
    source="user_message",
    priority=10,
    context="User is shopping for a high-performance laptop"
)

print(f"\nResolved budget: {llm_merge_ctx.data['budget']}")
print("\n→ LLM intelligently resolved the conflict")


LLM-Assisted Merging Example

Initial state:
Budget: around $1500

------------------------------------------------------------

User later says something that might conflict...
Using LLM to resolve: 'around $1500' vs '$1500-2000 range'

Resolved budget: $1500-2000 range

→ LLM intelligently resolved the conflict


The LLM-based resolution demonstrates semantic conflict handling:
- Conflict detection: `intelligent_merge()` checks if key already exists in self.data before merging.
- LLM invocation: Constructs prompt with old value, new value, and additional context, then calls `llm.invoke()`.
- Semantic analysis: LLM determines whether "around 1500" and "1500-2000 range" are contradictory (pick one) or complementary (combine).
- Intelligent combination: Instead of simple overwrite or rejection, LLM likely produces something like "$1500-2000 budget range" that captures both pieces of information.
- Metadata tracking: Result is stored with `strategy_used: 'llm_assisted'` for audit trail.

When to use LLM-assisted merging:
- Natural language user inputs that need semantic understanding.
- Ambiguous conflicts where rule-based strategies would lose information.
- Complementary information that should be combined rather than replaced.
- Values that require context to resolve properly.
- Cases where explainability matters ("why did you merge these this way?").

Trade-offs:
- Benefits: Handles semantic nuance, combines complementary info, contextually aware, more human-like.
- Drawbacks: Slower and more expensive than rule-based (LLM API call per conflict), non-deterministic (same conflict might resolve differently), requires careful prompt engineering, potential for hallucination.

This hybrid approach - rule-based strategies for structured data, LLM assistance for semantic conflicts - provides the best of both worlds for production context management.

## Part 5: Comparing strategies
Let's compare all four strategies side-by-side with the same scenario.

To truly understand the trade-offs between these four strategies, let's run them all through the exact same scenario - a user changing their budget from 1500 to 2000 - and observe how each strategy handles this simple update differently. By seeing them side by side with identical input, the distinctions become crystal clear. Append-only keeps both values in chronological order, update-in-place keeps only the final value, versioned keeps both as distinct versions with metadata, and structured merge tracks the provenance of each update. This comparison reveals not just what each strategy does, but why we would choose one over another for different types of data and use cases. Let's create instances of all four context types and put them through the same sequence of operations.

In [24]:
print("Strategy Comparison: Same Scenario")
print("="*60)
print("\nScenario: User changes budget from $1500 to $2000")
print("="*60)

# 1. Append-only
print("\n1. APPEND-ONLY STRATEGY:")
print("-" * 60)
append_ctx = AppendOnlyContext()  # Create append-only context
# Record both budget values as separate entries
append_ctx.append("preference", "Budget is $1500")
append_ctx.append("preference", "Budget is now $2000")
# Display full timeline
print(append_ctx.get_full_context())
print("\nPros: Complete history, can see change happened")
print("Cons: Agent must infer current budget is $2000")
print("Context size: Medium")

# 2. Update-in-place
print("\n" + "="*60)
print("\n2. UPDATE-IN-PLACE STRATEGY:")
print("-" * 60)
update_ctx = UpdateInPlaceContext()  # Create update-in-place context
update_ctx.set("budget", 1500)  # Set initial budget
update_ctx.set("budget", 2000)  # Update budget - overwrites the old value completely
print(update_ctx.get_context())  # Display current state (only latest value visible)
print("\nPros: Clean, current state only, no ambiguity")
print("Cons: Lost history, can't see it was $1500 before")
print("Context size: Minimal")

# 3. Versioned
print("\n" + "="*60)
print("\n3. VERSIONED STRATEGY:")
print("-" * 60)
version_ctx = VersionedContext()  # Create versioned context
version_ctx.set("budget", 1500, "Initial budget")  # Set initial budget (creates version 1)
version_ctx.set("budget", 2000, "Increased budget")  # Update budget (creates version 2, version 1 preserved)
# Display current state and history
print(f"Current: ${version_ctx.get_current('budget')}")
print(f"History: {len(version_ctx.get_history('budget'))} versions")
for v in version_ctx.get_history('budget'):
    print(f"  v{v.version}: ${v.value} - {v.reason}")
print("\nPros: Both current state AND history available")
print("Cons: More complex, need to decide what to include in context")
print("Context size: Large (if including history)")

# 4. Structured merge
print("\n" + "="*60)
print("\n4. STRUCTURED MERGE STRATEGY:")
print("-" * 60)
sm_ctx = StructuredMergeContext()  # Create structured merge context
sm_ctx.merge({"budget": 1500}, MergeStrategy.OVERWRITE, "user", 10)  # Set initial budget with provenance
sm_ctx.merge({"budget": 2000}, MergeStrategy.OVERWRITE, "user", 10)  # Update budget - old value overwritten but metadata tracked
# Display current value with metadata
print(f"Budget: ${sm_ctx.data['budget']}")
print(f"Source: {sm_ctx.metadata['budget']['source']}")
print(f"Strategy: {sm_ctx.metadata['budget']['strategy_used']}")
print("\nPros: Intelligent merging, tracks provenance")
print("Cons: Overhead of metadata, complexity")
print("Context size: Medium")

Strategy Comparison: Same Scenario

Scenario: User changes budget from $1500 to $2000

1. APPEND-ONLY STRATEGY:
------------------------------------------------------------
[14:27:30] preference: Budget is $1500
[14:27:30] preference: Budget is now $2000

Pros: Complete history, can see change happened
Cons: Agent must infer current budget is $2000
Context size: Medium


2. UPDATE-IN-PLACE STRATEGY:
------------------------------------------------------------
Current context:
  budget: 2000 (updated: 14:27:30)

Pros: Clean, current state only, no ambiguity
Cons: Lost history, can't see it was $1500 before
Context size: Minimal


3. VERSIONED STRATEGY:
------------------------------------------------------------
Current: $2000
History: 2 versions
  v1: $1500 - Initial budget
  v2: $2000 - Increased budget

Pros: Both current state AND history available
Cons: More complex, need to decide what to include in context
Context size: Large (if including history)


4. STRUCTURED MERGE STRATEGY:

Decision factors observed:
- If we need minimal context size → update-in-place wins (constant O(1) storage).
- If we need complete audit trail → append-only or versioned (full history).
- If we need both current + history → versioned (optimized for both access patterns).
- If we need multi-source integration → structured merge (provenance + conflict resolution).
- If simplicity is priority → update-in-place (just a dictionary).
- If compliance/debugging matters → append-only or versioned (nothing lost).

This comparison makes it clear that strategy selection is not about "best" but about matching requirements to capabilities.

## Decision Matrix for Strategy Selection

### 1. Use Case

#### Append-Only
* Chat conversation history
* Audit logs and compliance
* Event streams
* Debugging (need to see everything)

#### Update-In-Place
* User preferences (current only)
* Shopping cart (current state)
* Form data
* Real-time dashboards

#### Versioned
* User preferences (with history)
* Document editing
* Configuration management
* A/B testing tracking

#### Structured Merge
* Multi-source data integration
* Collaborative editing
* Conflict resolution needed
* Priority-based updates

### 2. Token Efficiency

| Strategy         | Token Efficiency                    |
| ---------------- | ----------------------------------- |
| Append-Only      | Low (grows indefinitely)            |
| Update-In-Place  | High (minimal size)                 |
| Versioned        | Medium (depends on what's included) |
| Structured Merge | Medium (includes metadata)          |

### 3. History Preservation

| Strategy         | History Preservation   |
| ---------------- | ---------------------- |
| Append-Only      | Complete               |
| Update-In-Place  | None                   |
| Versioned        | Complete with versions |
| Structured Merge | Partial (via metadata) |

### 4. Complexity

| Strategy         | Complexity |
| ---------------- | ---------- |
| Append-Only      | Low        |
| Update-In-Place  | Very Low   |
| Versioned        | Medium     |
| Structured Merge | High       |

### 5. Conflict Resolution

| Strategy         | Conflict Resolution          |
| ---------------- | ---------------------------- |
| Append-Only      | None (keeps all)             |
| Update-In-Place  | Simple (last write wins)     |
| Versioned        | Preserves all versions       |
| Structured Merge | Intelligent (strategy-based) |

### Quick Selection Guide
* **Need complete history?** → Append-Only or Versioned
* **Need minimal tokens?** → Update-In-Place
* **Need to merge from multiple sources?** → Structured Merge
* **Need both current state and history?** → Versioned
* **Just need current state?** → Update-In-Place
* **Need audit trail?** → Append-Only or Versioned
* **Need conflict resolution?** → Structured Merge

## Putting it all together: Hybrid approach
In real production systems, we rarely use just one strategy for all our context data. Different types of information have fundamentally different requirements - conversation history benefits from append-only preservation, current form state needs update-in-place efficiency, important user preferences deserve versioned tracking, and user profile data from multiple sources requires structured merging. The key insight is that these strategies are not mutually exclusive - we can and should use multiple strategies simultaneously, each applied to the type of data it handles best. A hybrid context manager combines all four approaches, routing each piece of information to the appropriate storage strategy based on its semantic meaning and access patterns. This gives us the advantages of all strategies while mitigating their individual weaknesses, at the cost of slightly increased complexity in the context manager itself. Let's build a comprehensive hybrid context manager that demonstrates how to combine these strategies effectively.

In [25]:
class HybridContextManager:
    """Production context manager using multiple strategies."""
    
    def __init__(self, llm):
        # Append-only for conversation - preserves complete chat history
        self.conversation = AppendOnlyContext()
        
        # Update-in-place for current state - minimal storage for ephemeral state
        self.current_state = UpdateInPlaceContext()
        
        # Versioned for important preferences - history + current value
        self.preferences = VersionedContext()
        
        # Structured merge for multi-source data - provenance + conflict resolution
        self.user_profile = StructuredMergeContext()
        
        self.llm = llm  # LLM instance for advanced operations
    
    def add_message(self, role: str, content: str):
        """
        Add to conversation history using append-only strategy.
        
        Args:
            role: Message role (user, agent, system)
            content: Message content
        """
        self.conversation.append(f"{role}_message", content)
    
    def update_state(self, key: str, value: Any):
        """
        Update current state using update-in-place strategy.
        
        Args:
            key: State variable name
            value: Current value
        """
        self.current_state.set(key, value)
    
    def set_preference(self, key: str, value: Any, reason: str = None):
        """
        Set versioned preference - preserves history.
        
        Args:
            key: Preference name
            value: Preference value
            reason: Optional explanation for this change
        """
        self.preferences.set(key, value, reason)
    
    def merge_profile_data(self, data: Dict, source: str, priority: int):
        """
        Merge user profile data using structured merge strategy.
        
        Args:
            data: Profile fields to merge
            source: Data source identifier
            priority: Source priority level
        """
        self.user_profile.merge(data, MergeStrategy.TAKE_HIGHER_PRIORITY, source, priority)
    
    def get_context_for_agent(self, include_full_history: bool = False) -> str:
        """
        Get complete context for agent, combining all strategies.
        
        Args:
            include_full_history: Whether to include full conversation or recent only
            
        Returns:
            Formatted context string optimized for agent consumption
        """
        parts = []
        
        # Conversation (recent only for efficiency, unless full requested)
        parts.append("## Recent Conversation")
        if include_full_history:
            parts.append(self.conversation.get_full_context())
        else:
            parts.append(self.conversation.get_recent_context(n=5))  # Last 5 messages
        
        # Current state (always minimal)
        parts.append("\n## Current State")
        parts.append(self.current_state.get_context())
        
        # Preferences (current only, history available if needed)
        parts.append("\n## User Preferences")
        parts.append(self.preferences.get_current_context())
        
        # User profile (with provenance)
        parts.append("\n## User Profile")
        parts.append(self.user_profile.get_context())
        
        return "\n".join(parts)

The `HybridContextManager` demonstrates production-ready multi-strategy context management:
- Four specialized stores: Each type of context routed to appropriate storage strategy.
- Semantic routing: Methods like `add_message()`, `update_state()`, `set_preference()` and `merge_profile_data()` automatically use the right strategy.
- Unified retrieval: `get_context_for_agent()` combines all four sources into coherent context string.
- Efficiency controls: Can choose between full history or recent-only for conversation to manage token usage.

The genius of this approach is that each data type gets optimal handling without the caller needing to know about strategies - the interface is simple while the implementation is sophisticated. Now let's see this in action with a realistic shopping session.

In [26]:
# Example: Production usage
print("Hybrid Context Manager: Production Example")
print("="*60)

# Create hybrid context manager for production usage
manager = HybridContextManager(llm)

# Simulate a session
print("\nSimulating a shopping session...\n")

# Step 1: Record conversation using append-only (complete history)
manager.add_message("user", "I'm looking for a laptop")
manager.add_message("agent", "I can help! What's your budget?")
manager.add_message("user", "Around $1500")

# Step 2: Set preference using versioned strategy (history + current)
manager.set_preference("budget", 1500, "User stated budget")

# Step 3: Update current state using update-in-place (ephemeral state)
manager.update_state("search_category", "laptops")
manager.update_state("items_in_cart", 0)  # Will be updated as cart changes

# Step 4: Merge profile data from multiple sources using structured merge
# Data from database (lower priority)
manager.merge_profile_data(
    {"name": "Alice", "member_since": "2023"},
    source="database",
    priority=5
)

# Data from direct user input (higher priority)
manager.merge_profile_data(
    {"email": "alice@example.com"},
    source="user_input",
    priority=10
)

# Step 5: User changes mind - each strategy handles appropriately
manager.add_message("user", "Actually, I can go up to $2000")  # Appended to conversation
manager.set_preference("budget", 2000, "User increased budget")  # Creates version 2
manager.update_state("search_category", "gaming_laptops")  # Overwrites old value
manager.update_state("items_in_cart", 1)  # Updates cart count

# Get context for agent - combines all four strategies into unified view
print("Context provided to agent:")
print("="*60)
print(manager.get_context_for_agent(include_full_history=False))

Hybrid Context Manager: Production Example

Simulating a shopping session...

Context provided to agent:
## Recent Conversation
[14:36:21] user_message: I'm looking for a laptop
[14:36:21] agent_message: I can help! What's your budget?
[14:36:21] user_message: Around $1500
[14:36:21] user_message: Actually, I can go up to $2000

## Current State
Current context:
  search_category: gaming_laptops (updated: 14:36:21)
  items_in_cart: 1 (updated: 14:36:21)

## User Preferences
Current state:
  budget: 2000 (v2)

## User Profile
Current context:
  name: Alice
    (source: database, priority: 5)
  member_since: 2023
    (source: database, priority: 5)
  email: alice@example.com
    (source: user_input, priority: 10)


The hybrid manager routed each update to the appropriate strategy automatically:
- Conversation messages: Stored in `AppendOnlyContext` - full history preserved including both budget mentions.
- Budget preference: Stored in `VersionedContext` - version 1 (1500) and version 2 (2000) both available.
- Search category and cart count: Stored in `UpdateInPlaceContext` - only current values kept (gaming_laptops, 1).
- User profile: Stored in `StructuredMergeContext` - name (database, priority 5), email (user_input, priority 10).

Production considerations:
- Total context size is sum of all four sources - monitor token usage.
- Can selectively include/exclude sections based on task needs.
- Conversation can be truncated to recent N messages for efficiency.
- Preference and profile history available but not automatically included in agent context.
- Each strategy's benefits apply to appropriate data types without caller complexity.

This architecture scales to production by matching storage strategy to data semantics rather than forcing one-size-fits-all.